In [11]:
!pip install tensorflow_decision_forests
!pip install wurlitzer


In [12]:
import pandas as pd
import numpy as np
import tensorflow_decision_forests as tfdf
import tensorflow as tf

In [13]:
dublin_port = pd.read_csv('dublin_port.csv')
dublin_port.head()

,Unnamed: 0,datetime,rain,temp,wetb,dewpt,vappr,rhum,msl,no2,so2,pm10,pm2.5
0,0,2022-01-01 00:00:00,0.0,13.1,11.4,9.8,12.1,80.0,1007.3,22.00,2.12,7.31,4.08
1,1,2022-01-01 01:00:00,0.0,13.9,11.2,8.5,11.1,70.0,1005.8,17.49,2.05,20.61,12.70
2,2,2022-01-01 02:00:00,0.0,14.0,11.0,8.1,10.8,67.0,1005.0,13.28,2.29,30.00,16.92
3,3,2022-01-01 03:00:00,0.0,14.6,11.4,8.4,11.0,66.0,1003.9,18.56,2.79,31.65,17.93
4,4,2022-01-01 04:00:00,0.0,14.9,12.1,9.5,11.9,70.0,1002.7,13.83,2.07,27.55,15.19


In [14]:
dublin_port = dublin_port.iloc[:, ~dublin_port.columns.isin(['Unnamed: 0', 'datetime'])]

total_samples = len(dublin_port)
train_samples = int(0.7 * total_samples)
val_samples = int(0.15 * total_samples)

train_data = dublin_port[:train_samples]
val_data = dublin_port[train_samples:train_samples + val_samples]
test_data = dublin_port[train_samples + val_samples:]

print("Training:", len(train_data))
print("Validation:", len(val_data))
print("Testing:", len(test_data))

Training: 6115
Validation: 1310
Testing: 1312


In [15]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_data, label="pm2.5", task=tfdf.keras.Task.REGRESSION)
val_ds = tfdf.keras.pd_dataframe_to_tf_dataset(val_data, label="pm2.5", task=tfdf.keras.Task.REGRESSION)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_data, label="pm2.5", task=tfdf.keras.Task.REGRESSION)

In [16]:
model1 = tfdf.keras.GradientBoostedTreesModel(task = tfdf.keras.Task.REGRESSION)

Use /tmp/tmpjyx9v1ru as temporary training directory


In [17]:
model1.fit(train_ds)

Reading training dataset...
Training dataset read in 0:00:00.644108. Found 6115 examples.
Training model...
Model trained in 0:00:04.783546
Compiling model...
Model compiled.


In [18]:
model1.compile(metrics=["mse", "mae"])
model1_val_result = model1.evaluate(val_ds, return_dict=True)

2/2 [==============================] - 0s 18ms/step - loss: 0.0000e+00 - mse: 5.6360 - mae: 1.6605


In [19]:
model1_val_result

{'loss': 0.0, 'mse': 5.636040210723877, 'mae': 1.6604976654052734}

In [20]:
tuner = tfdf.tuner.RandomSearch(num_trials=50)
tuner.choice("min_examples", [2, 5, 7, 10])
tuner.choice("categorical_algorithm", ["CART", "RANDOM"])
local_search_space = tuner.choice("growing_strategy", ["LOCAL"])
local_search_space.choice("max_depth", [3, 4, 5, 6, 8])
global_search_space = tuner.choice("growing_strategy", ["BEST_FIRST_GLOBAL"], merge=True)
global_search_space.choice("max_num_nodes", [16, 32, 64, 128, 256])
tuner.choice("num_candidate_attributes_ratio", [0.2, 0.5, 0.9, 1.0])

In [21]:
tuned_model = tfdf.keras.GradientBoostedTreesModel(task = tfdf.keras.Task.REGRESSION, tuner=tuner)
tuned_model.fit(train_ds, verbose=2)

Use /tmp/tmpqxipp5s5 as temporary training directory
Reading training dataset...
Training tensor examples:
Features: {'rain': <tf.Tensor 'data:0' shape=(None,) dtype=float64>, 'temp': <tf.Tensor 'data_1:0' shape=(None,) dtype=float64>, 'wetb': <tf.Tensor 'data_2:0' shape=(None,) dtype=float64>, 'dewpt': <tf.Tensor 'data_3:0' shape=(None,) dtype=float64>, 'vappr': <tf.Tensor 'data_4:0' shape=(None,) dtype=float64>, 'rhum': <tf.Tensor 'data_5:0' shape=(None,) dtype=float64>, 'msl': <tf.Tensor 'data_6:0' shape=(None,) dtype=float64>, 'no2': <tf.Tensor 'data_7:0' shape=(None,) dtype=float64>, 'so2': <tf.Tensor 'data_8:0' shape=(None,) dtype=float64>, 'pm10': <tf.Tensor 'data_9:0' shape=(None,) dtype=float64>}
Label: Tensor("data_10:0", shape=(None,), dtype=float64)
Weights: None
Normalized tensor features:
 {'rain': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'Cast:0' shape=(None,) dtype=float32>), 'temp': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.T

[INFO 23-07-11 21:16:12.1732 UTC kernel.cc:773] Start Yggdrasil model training
[INFO 23-07-11 21:16:12.1733 UTC kernel.cc:774] Collect training examples
[INFO 23-07-11 21:16:12.1733 UTC kernel.cc:787] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: NUMERICAL
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

[INFO 23-07-11 21:16:12.1734 UTC kernel.cc:393] Number of batches: 7
[INFO 23-07-11 21:16:12.1734 UTC kernel.cc:394] Number of examples: 6115
[INFO 23-07-11 21:16:12.1746 UTC kernel.cc:794] Training dataset:
Number of records: 6115
Number of columns: 11

Number of columns by type:
	NUMERICAL: 11 (100%)

Columns:

NUMERICAL: 11 (100%)
	0: "__LABEL" NUMERICAL mean:6.97927 min:0.51 max:19.2 sd:3.63809
	1: "dewpt" NUMERICAL mean:7.47856 min:-5 max:18.8 sd:4.52343
	2: "msl" NUMERICAL mean:1017.38 mi

Model trained in 0:02:35.898436
Compiling model...
Model compiled.


In [22]:
tuned_model.compile(metrics=["mse", "mae"])
tuned_model_val_result = tuned_model.evaluate(val_ds, return_dict=True)

2/2 [==============================] - 0s 13ms/step - loss: 0.0000e+00 - mse: 6.0137 - mae: 1.7160


In [23]:
tuned_model_val_result

{'loss': 0.0, 'mse': 6.013693332672119, 'mae': 1.7160286903381348}

In [24]:
tuning_logs = tuned_model.make_inspector().tuning_logs()
tuning_logs

,score,evaluation_time,best,min_examples,categorical_algorithm,growing_strategy,max_num_nodes,num_candidate_attributes_ratio,max_depth
0,-1.960090,4.521754,False,2,RANDOM,BEST_FIRST_GLOBAL,32.0,0.5,NaN
1,-1.998669,11.226706,False,7,RANDOM,LOCAL,NaN,0.5,6.0
2,-1.911054,15.357245,False,7,CART,BEST_FIRST_GLOBAL,256.0,1.0,NaN
3,-1.947484,25.426311,False,10,CART,LOCAL,NaN,0.5,6.0
4,-1.907538,29.119574,False,2,CART,BEST_FIRST_GLOBAL,256.0,0.9,NaN
5,-2.128587,29.567264,False,7,CART,LOCAL,NaN,0.5,3.0
6,-1.983955,34.705446,False,10,CART,BEST_FIRST_GLOBAL,16.0,1.0,NaN
7,-1.958282,35.307267,False,10,CART,BEST_FIRST_GLOBAL,64.0,0.9,NaN
8,-2.127177,36.437739,False,2,CART,LOCAL,NaN,0.5,3.0
9,-1.978377,39.198208,False,7,CART,BEST_FIRST_GLOBAL,128.0,0.5,NaN


In [25]:
auto_tuner = tfdf.tuner.RandomSearch(num_trials=50, use_predefined_hps=True)

In [26]:
auto_tuned_model = tfdf.keras.GradientBoostedTreesModel(task = tfdf.keras.Task.REGRESSION, tuner=auto_tuner)
auto_tuned_model.fit(train_ds, verbose=2)

Use /tmp/tmpowa6ij70 as temporary training directory
Reading training dataset...
Training tensor examples:
Features: {'rain': <tf.Tensor 'data:0' shape=(None,) dtype=float64>, 'temp': <tf.Tensor 'data_1:0' shape=(None,) dtype=float64>, 'wetb': <tf.Tensor 'data_2:0' shape=(None,) dtype=float64>, 'dewpt': <tf.Tensor 'data_3:0' shape=(None,) dtype=float64>, 'vappr': <tf.Tensor 'data_4:0' shape=(None,) dtype=float64>, 'rhum': <tf.Tensor 'data_5:0' shape=(None,) dtype=float64>, 'msl': <tf.Tensor 'data_6:0' shape=(None,) dtype=float64>, 'no2': <tf.Tensor 'data_7:0' shape=(None,) dtype=float64>, 'so2': <tf.Tensor 'data_8:0' shape=(None,) dtype=float64>, 'pm10': <tf.Tensor 'data_9:0' shape=(None,) dtype=float64>}
Label: Tensor("data_10:0", shape=(None,), dtype=float64)
Weights: None
Normalized tensor features:
 {'rain': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'Cast:0' shape=(None,) dtype=float32>), 'temp': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.T

[INFO 23-07-11 21:19:34.7012 UTC kernel.cc:773] Start Yggdrasil model training
[INFO 23-07-11 21:19:34.7012 UTC kernel.cc:774] Collect training examples
[INFO 23-07-11 21:19:34.7013 UTC kernel.cc:787] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: NUMERICAL
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

[INFO 23-07-11 21:19:34.7014 UTC kernel.cc:393] Number of batches: 7
[INFO 23-07-11 21:19:34.7014 UTC kernel.cc:394] Number of examples: 6115
[INFO 23-07-11 21:19:34.7022 UTC kernel.cc:794] Training dataset:
Number of records: 6115
Number of columns: 11

Number of columns by type:
	NUMERICAL: 11 (100%)

Columns:

NUMERICAL: 11 (100%)
	0: "__LABEL" NUMERICAL mean:6.97927 min:0.51 max:19.2 sd:3.63809
	1: "dewpt" NUMERICAL mean:7.47856 min:-5 max:18.8 sd:4.52343
	2: "msl" NUMERICAL mean:1017.38 mi

Model trained in 0:36:19.303861
Compiling model...
Model compiled.


In [27]:
auto_tuned_model.compile(metrics=["mse", "mae"])
auto_tuned_model_val_result = auto_tuned_model.evaluate(val_ds, return_dict=True)

2/2 [==============================] - 0s 20ms/step - loss: 0.0000e+00 - mse: 5.7791 - mae: 1.6832


In [28]:
auto_tuned_model_val_result

{'loss': 0.0, 'mse': 5.779103755950928, 'mae': 1.6832051277160645}

In [30]:
auto_tuning_logs = auto_tuned_model.make_inspector().tuning_logs()
auto_tuning_logs

,score,evaluation_time,best,split_axis,sparse_oblique_projection_density_factor,sparse_oblique_normalization,sparse_oblique_weights,categorical_algorithm,growing_strategy,max_num_nodes,sampling_method,subsample,shrinkage,min_examples,num_candidate_attributes_ratio,max_depth
0,-1.917802,90.560487,False,SPARSE_OBLIQUE,5.0,MIN_MAX,BINARY,CART,BEST_FIRST_GLOBAL,32.0,RANDOM,1.0,0.10,10,1.0,NaN
1,-1.903895,120.072636,False,SPARSE_OBLIQUE,4.0,NONE,BINARY,CART,BEST_FIRST_GLOBAL,512.0,RANDOM,1.0,0.05,5,0.9,NaN
2,-2.096056,152.836276,False,SPARSE_OBLIQUE,4.0,STANDARD_DEVIATION,CONTINUOUS,CART,LOCAL,NaN,RANDOM,0.8,0.05,7,1.0,3.0
3,-1.896099,222.915008,False,SPARSE_OBLIQUE,4.0,STANDARD_DEVIATION,BINARY,CART,LOCAL,NaN,RANDOM,1.0,0.02,20,1.0,8.0
4,-1.909813,269.017294,False,SPARSE_OBLIQUE,5.0,MIN_MAX,BINARY,RANDOM,BEST_FIRST_GLOBAL,64.0,RANDOM,0.9,0.10,20,0.2,NaN
5,-1.955130,289.132869,False,SPARSE_OBLIQUE,5.0,MIN_MAX,BINARY,CART,BEST_FIRST_GLOBAL,16.0,RANDOM,0.6,0.10,20,0.5,NaN
6,-1.915706,325.080889,False,SPARSE_OBLIQUE,1.0,NONE,CONTINUOUS,RANDOM,BEST_FIRST_GLOBAL,256.0,RANDOM,0.6,0.10,10,0.9,NaN
7,-1.860066,387.627638,False,SPARSE_OBLIQUE,2.0,MIN_MAX,BINARY,RANDOM,BEST_FIRST_GLOBAL,512.0,RANDOM,0.6,0.05,7,0.9,NaN
8,-1.854117,396.206714,False,SPARSE_OBLIQUE,4.0,STANDARD_DEVIATION,CONTINUOUS,CART,BEST_FIRST_GLOBAL,64.0,RANDOM,0.9,0.05,20,0.2,NaN
9,-1.902129,474.923737,False,SPARSE_OBLIQUE,5.0,NONE,CONTINUOUS,CART,BEST_FIRST_GLOBAL,64.0,RANDOM,0.6,0.05,10,0.2,NaN
